In [1]:
!pip install gymnasium

In [2]:
import gymnasium as gym
import numpy as np
import time
from IPython import display

In [3]:
env = gym.make('FrozenLake-v1', render_mode="ansi")

In [4]:
env.unwrapped.P[0][3] # Transition model

[(0.3333333333333333, 1, 0.0, False),
 (0.3333333333333333, 0, 0.0, False),
 (0.3333333333333333, 0, 0.0, False)]

In [5]:
env.observation_space.n

16

In [6]:
env.action_space.n

4

In [7]:
def play(env, policy, render=False):
    state, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info, _ = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            print(env.render())
            time.sleep(0.5)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [8]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0)

(0.0, 12)

In [9]:

policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play(env, policy_0, True)


  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 8)

In [10]:
reward, step = 0, None
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
reward, step = play(env, policy_1, True)
print(reward, step)

  (Up)
SFFF
FHFH
FFFH
HFFG

0.0 8


In [11]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play(env, policy_2, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(0.0, 4)

In [12]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play(env, policy_3, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 21)

In [13]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    # print(f'Number of successes: {success}/{max_episodes}')
    # print(f'Average number of steps: {np.mean(list_of_steps)}')
    return (success, np.mean(list_of_steps))

In [14]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
play_multiple_times(env, policy_0, 1000)

c:\Users\Legion 5 Pro\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Legion 5 Pro\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


(0, nan)

In [15]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
play_multiple_times(env, policy_1, 1000)

(51, 11.137254901960784)

In [16]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
play_multiple_times(env, policy_2, 1000)

(106, 14.971698113207546)

In [17]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
play_multiple_times(env, policy_3, 1000)

(777, 43.88030888030888)

In [18]:
epsilon = 1e-9

In [19]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9, bool_print = True):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.unwrapped.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value

        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            if bool_print:
                print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [20]:
policy_0 = np.asarray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
v_values_0 = policy_evaluation(env, policy_0)
print(v_values_0)

Converged at 0-th iteration.
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [21]:
policy_1 = np.asarray([0, 1, 1, 3, 1, 0, 2, 0, 1, 1, 2, 2, 3, 3, 1, 0])
v_values_1 = policy_evaluation(env, policy_1)
print(v_values_1)

Converged at 48-th iteration.
[0.01904157 0.01519815 0.03161906 0.02371389 0.02538879 0.
 0.06648515 0.         0.05924054 0.13822794 0.18999823 0.
 0.         0.21152109 0.56684236 0.        ]


In [22]:
np.all(v_values_1 >= v_values_0)

True

In [23]:
policy_2 = np.array([1, 1, 1, 3, 0, 1, 2, 3, 1, 1, 2, 3, 2, 2, 1, 3])
v_values_2 = policy_evaluation(env, policy_2)
print(v_values_2)

Converged at 53-th iteration.
[0.02889625 0.01951972 0.03616977 0.0271268  0.04790519 0.
 0.07391985 0.         0.08288277 0.19339319 0.21022995 0.
 0.         0.35153135 0.62684674 0.        ]


In [24]:
np.all(v_values_2 >= v_values_1)

True

In [25]:
policy_3 = np.array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])
v_values_3 = policy_evaluation(env, policy_3)
print(v_values_3)

Converged at 80-th iteration.
[0.06888666 0.06141097 0.07440714 0.05580443 0.09185068 0.
 0.11220679 0.         0.14543323 0.24749485 0.29961611 0.
 0.         0.37993438 0.63901935 0.        ]


In [26]:
np.all(v_values_3 >= v_values_2)

True

In [27]:
def value_iteration(env, max_iters=500, gamma=0.9, bool_print = True):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []

            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.unwrapped.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])

                q_values.append(q_value)

            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]

        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            if bool_print:
                print(f'Converged at {i}-th iteration.')
            break

    return v_values

In [28]:
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)

Converged at 79-th iteration.


In [29]:
optimal_v_values

array([0.06888615, 0.06141054, 0.07440682, 0.05580409, 0.09185022,
       0.        , 0.11220663, 0.        , 0.14543286, 0.2474946 ,
       0.29961593, 0.        , 0.        , 0.3799342 , 0.63901926,
       0.        ])

In [30]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int32)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.unwrapped.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])

            q_values.append(q_value)

        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action

    return policy

In [31]:
optimal_policy = policy_extraction(env, optimal_v_values, gamma=0.9)

In [32]:
optimal_policy

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])

In [33]:
play(env, optimal_policy, True)

  (Left)
SFFF
FHFH
FFFH
HFFG



(0.0, 102)

In [34]:
play_multiple_times(env, optimal_policy, 1000)

(791, 44.007585335018966)

In [35]:
def policy_iteration(env, max_iters=500, gamma=0.9, bool_print = True):
    # Initialize a random policy
    policy = np.random.randint(0, env.action_space.n, env.observation_space.n)
    print("Initialize a random policy:", policy)

    for i in range(max_iters):
        # Policy Evaluation
        v_values = policy_evaluation(env, policy, max_iters, gamma, False)

        # Policy Improvement
        new_policy = policy_extraction(env, v_values, gamma)

        # Check if the policy has converged
        if np.array_equal(policy, new_policy):
            if bool_print:
                print(f'Policy is converged at {i}-th iteration.')
            break
        else:
            policy = new_policy
    
    return policy


In [36]:
optimal_policy_pi = policy_iteration(env, 100, 0.9)

Initialize a random policy: [0 3 2 1 3 3 2 1 3 3 1 0 2 3 1 2]
Policy is converged at 3-th iteration.


In [37]:
optimal_policy_pi

array([0, 3, 0, 3, 0, 0, 0, 0, 3, 1, 0, 0, 0, 2, 1, 0])

In [38]:
play(env, optimal_policy_pi, True)

  (Down)
SFFF
FHFH
FFFH
HFFG



(1.0, 65)

In [39]:
play_multiple_times(env, optimal_policy_pi, 10000)

(7862, 43.39646400407021)

# Experiment

### Experiment with FrozenLake-v1

In [40]:
num_trials_for_FrozenLake = 1000000
num_trials_for_Taxi = 2000000

In [41]:
env_1 = gym.make('FrozenLake-v1')

In [42]:
time_find_optimal_policy_start = time.time()
vi_value = value_iteration(env_1, 500, 0.9)
policy_from_vi_value = policy_extraction(env_1, vi_value, 0.9)
time_find_optimal_policy_end = time.time() - time_find_optimal_policy_start
print("Policy: ", policy_from_vi_value, "\n")
start_vi = time.time()
vi_number_of_successes, vi_mean_of_steps = play_multiple_times(env_1, policy_from_vi_value, num_trials_for_FrozenLake)
vi_time = time.time() - start_vi
print(f'Number of successes of Value Iteration in map FrozenLake-v1 : {vi_number_of_successes}/{num_trials_for_FrozenLake}, Time to find policy: {time_find_optimal_policy_end}, Total time: {vi_time}s, Average time : {vi_time/num_trials_for_FrozenLake}s, Average steps : {vi_mean_of_steps} steps')

Converged at 79-th iteration.
Policy:  [0 3 0 3 0 0 0 0 3 1 0 0 0 2 1 0] 

Number of successes of Value Iteration in map FrozenLake-v1 : 780719/1000000, Time to find policy: 0.020755290985107422, Total time: 295.8431067466736s, Average time : 0.0002958431067466736s, Average steps : 43.57424118024539 steps


In [43]:
time_find_optimal_policy_start = time.time()
policy_pi = policy_iteration(env_1, 500, 0.9)
time_find_optimal_policy_end = time.time() - time_find_optimal_policy_start
print("Policy: ", policy_pi, "\n")
start_pi = time.time()
pi_number_of_successes, pi_mean_of_steps = play_multiple_times(env_1, policy_pi, num_trials_for_FrozenLake)
pi_time = time.time() - start_pi
print(f'Number of successes of Policy Iteration in map FrozenLake-v1 : {pi_number_of_successes}/{num_trials_for_FrozenLake}, Time to find policy: {time_find_optimal_policy_end}, Total time: {pi_time}s, Average time : {pi_time/num_trials_for_FrozenLake}s, Average steps : {pi_mean_of_steps} steps')

Initialize a random policy: [0 1 2 3 3 3 2 1 3 0 2 3 3 2 2 3]
Policy is converged at 3-th iteration.
Policy:  [0 3 0 3 0 0 0 0 3 1 0 0 0 2 1 0] 

Number of successes of Policy Iteration in map FrozenLake-v1 : 780993/1000000, Time to find policy: 0.014512777328491211, Total time: 301.03059577941895s, Average time : 0.00030103059577941895s, Average steps : 43.49639369366947 steps


### Experiment with FrozenLake8x8-v1

In [44]:
env_2 = gym.make('FrozenLake8x8-v1')

In [45]:
time_find_optimal_policy_start = time.time()
vi_value = value_iteration(env_2, 500, 0.9)
policy_from_vi_value = policy_extraction(env_2, vi_value, 0.9)
time_find_optimal_policy_end = time.time() - time_find_optimal_policy_start
print("Policy: ", policy_from_vi_value, "\n")
start_vi = time.time()
vi_number_of_successes, vi_mean_of_steps = play_multiple_times(env_2, policy_from_vi_value, num_trials_for_FrozenLake)
vi_time = time.time() - start_vi
print(f'Number of successes of Value Iteration in map FrozenLake8x8-v1 : {vi_number_of_successes}/{num_trials_for_FrozenLake}, Time to find policy: {time_find_optimal_policy_end}, Total time: {vi_time}s, Average time : {vi_time/num_trials_for_FrozenLake}s, Average steps : {vi_mean_of_steps} steps')

Converged at 117-th iteration.
Policy:  [3 2 2 2 2 2 2 2 3 3 3 3 2 2 2 1 3 3 0 0 2 3 2 1 3 3 3 1 0 0 2 1 3 3 0 0 2
 1 3 2 0 0 0 1 3 0 0 2 0 0 1 0 0 0 0 2 0 1 0 0 1 1 1 0] 

Number of successes of Value Iteration in map FrozenLake8x8-v1 : 748864/1000000, Time to find policy: 0.08572936058044434, Total time: 502.4876980781555s, Average time : 0.0005024876980781555s, Average steps : 74.54548756516537 steps


In [46]:
time_find_optimal_policy_start = time.time()
policy_pi = policy_iteration(env_2, 500, 0.9)
time_find_optimal_policy_end = time.time() - time_find_optimal_policy_start
print("Policy: ", policy_pi, "\n")
start_pi = time.time()
pi_number_of_successes, pi_mean_of_steps = play_multiple_times(env_2, policy_pi, num_trials_for_FrozenLake)
pi_time = time.time() - start_pi
print(f'Number of successes of Policy Iteration in map FrozenLake8x8-v1 : {pi_number_of_successes}/{num_trials_for_FrozenLake}, Time to find policy: {time_find_optimal_policy_end}, Total time: {pi_time}s, Average time : {pi_time/num_trials_for_FrozenLake}s, Average steps : {pi_mean_of_steps} steps')

Initialize a random policy: [1 1 1 0 0 1 0 1 0 0 3 2 0 0 3 3 0 2 3 0 1 0 2 1 2 1 0 1 2 2 1 3 2 2 3 0 1
 1 1 0 1 0 3 0 3 0 3 1 1 3 1 3 3 2 0 3 3 2 1 3 0 2 1 1]
Policy is converged at 4-th iteration.
Policy:  [3 2 2 2 2 2 2 2 3 3 3 3 2 2 2 1 3 3 0 0 2 3 2 1 3 3 3 1 0 0 2 1 3 3 0 0 2
 1 3 2 0 0 0 1 3 0 0 2 0 0 1 0 0 0 0 2 0 1 0 0 1 1 1 0] 

Number of successes of Policy Iteration in map FrozenLake8x8-v1 : 749386/1000000, Time to find policy: 0.07019329071044922, Total time: 538.7594585418701s, Average time : 0.0005387594585418701s, Average steps : 74.53192480243827 steps


### Experiment with Taxi-v3

In [47]:
env_3 = gym.make("Taxi-v3")

In [48]:
time_find_optimal_policy_start = time.time()
vi_value = value_iteration(env_3, 500, 0.9)
policy_from_vi_value = policy_extraction(env_3, vi_value, 0.9)
time_find_optimal_policy_end = time.time() - time_find_optimal_policy_start
print("Policy: ", policy_from_vi_value, "\n")
start_vi = time.time()
vi_number_of_successes, vi_mean_of_steps = play_multiple_times(env_3, policy_from_vi_value, num_trials_for_Taxi)
vi_time = time.time() - start_vi
print(f'Number of successes of Value Iteration in map Taxi-v3 : {vi_number_of_successes}/{num_trials_for_Taxi}, Time to find policy: {time_find_optimal_policy_end}, Total time: {vi_time}s, Average time : {vi_time/num_trials_for_Taxi}s, Average steps : {vi_mean_of_steps} steps')

Converged at 116-th iteration.
Policy:  [4 4 4 4 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 3 3 3 3 0 0 0 0 0 0 0 0 0 0 0 0 3
 0 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 2 2 2 2 0 0 0 0 0 0
 0 0 0 2 0 0 0 0 0 0 4 4 4 4 0 0 0 0 0 0 0 0 0 5 0 0 1 1 1 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 1 1
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1
 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 2 2 2 2 0 0 0 0 2 2 2 2 1 2 0 2 1 1
 1 1 2 2 2 2 3 3 3 3 2 2 2 2 1 2 3 2 3 3 3 3 1 1 1 1 3 3 3 3 2 2 2 2 3 1 3
 2 3 3 3 3 1 1 1 1 3 3 3 3 0 0 0 0 3 1 3 0 3 3 3 3 1 1 1 1 3 3 3 3 0 0 0 0
 3 1 3 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1
 1 4 4 4 4 1 1 1 1 1 1 5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 4

In [49]:
time_find_optimal_policy_start = time.time()
policy_pi = policy_iteration(env_3, 500, 0.9)
time_find_optimal_policy_end = time.time() - time_find_optimal_policy_start
print("Policy: ", policy_pi, "\n")
start_pi = time.time()
pi_number_of_successes, pi_mean_of_steps = play_multiple_times(env_3, policy_pi, num_trials_for_Taxi)
pi_time = time.time() - start_pi
print(f'Number of successes of Policy Iteration in map Taxi-v3 : {pi_number_of_successes}/{num_trials_for_Taxi}, Time to find policy: {time_find_optimal_policy_end}, Total time: {pi_time}s, Average time : {pi_time/num_trials_for_Taxi}s, Average steps : {pi_mean_of_steps} steps')

Initialize a random policy: [3 5 1 4 0 3 0 5 2 2 1 2 4 5 2 5 1 2 1 2 0 5 5 0 1 0 4 5 3 4 4 5 5 0 0 3 2
 2 0 0 1 5 3 0 4 5 0 0 2 1 2 2 1 0 4 4 5 5 5 4 5 3 2 5 5 0 1 0 1 1 4 1 0 1
 4 0 2 0 2 0 1 0 0 5 4 1 4 0 1 2 3 2 2 2 0 5 3 2 1 4 4 4 1 5 2 3 4 4 1 4 3
 4 0 3 2 2 0 2 4 3 2 3 3 2 1 5 0 2 4 5 3 2 0 3 1 5 0 1 4 2 5 2 3 4 1 2 1 3
 1 5 2 3 1 4 1 3 1 5 5 5 5 5 2 0 5 3 1 1 5 3 4 1 2 4 4 3 3 0 3 5 4 1 0 0 4
 0 1 5 1 4 4 2 2 5 5 5 0 2 1 2 2 5 5 5 5 3 3 3 2 4 4 1 1 3 2 0 2 3 2 1 3 4
 3 1 2 3 2 4 1 0 2 0 1 5 3 0 4 1 5 1 3 2 3 3 4 1 4 5 1 4 1 1 2 0 2 1 1 2 0
 3 5 4 4 4 0 4 1 1 0 2 3 2 2 0 3 0 5 1 2 5 2 1 0 2 4 2 5 2 5 0 2 4 1 4 0 4
 1 2 2 1 4 4 5 0 1 1 3 0 5 2 5 4 5 4 5 2 0 5 5 1 5 4 3 3 2 2 4 5 2 4 1 5 5
 4 2 5 1 0 1 2 3 5 1 1 3 0 4 4 2 2 5 4 2 4 3 2 5 4 2 4 2 3 4 4 4 3 2 5 3 3
 4 1 1 5 1 1 4 0 0 0 3 1 2 1 4 1 1 2 5 2 3 2 5 5 2 5 5 3 1 2 0 2 2 5 4 1 4
 2 3 4 2 3 3 0 1 5 1 5 1 3 3 5 2 5 1 0 1 5 1 4 5 1 5 1 2 5 1 0 0 4 5 2 0 4
 5 2 0 3 2 4 4 4 5 5 0 0 2 2 3 0 1 3 4 5 4 5 5 4 2 0 5 4 3 2 0 0 4 4 3 0

# Conclusion